## [Previous Section](./ClassesDecorator.ipynb)

## Enhancing Classes with a Class Decorator

In [7]:
from collections.abc import Callable  
from typing import Any, NoReturn, get_type_hints

class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:  # <4>
            value = self.constructor()
        else:
            try:
                value = self.constructor(value) 
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = (
                    f'{value!r} is not compatible with {self.name}:{type_name}'
                )
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value

Every top-level function in this file is prefixed with an underscore, **except** the `checked` decorator. 

The methods to be injected in the decorated class. 

In [8]:
def _fields(cls: type) -> dict[str, type]:
    return get_type_hints(cls)

def __init__(self: Any, **kwargs: Any) -> None:
    for name in self._fields():
        value = kwargs.pop(name, ...)
        setattr(self, name, value)
    if kwargs:
        self.__flag_unknown_attrs(*kwargs)

def __setattr__(self: Any, name: str, value: Any) -> None:
    if name in self._fields():
        cls = self.__class__
        descriptor = getattr(cls, name)
        descriptor.__set__(self, value)
    else:
        self.__flag_unknown_attrs(name)

def __flag_unknown_attrs(self: Any, *names: str) -> NoReturn:
    plural = 's' if len(names) > 1 else ''
    extra = ', '.join(f'{name!r}' for name in names)
    cls_name = repr(self.__class__.__name__)
    raise AttributeError(f'{cls_name} has no attribute{plural} {extra}')

def _asdict(self: Any) -> dict[str, Any]:
    return {
        name: getattr(self, name)
        for name, attr in self.__class__.__dict__.items()
        if isinstance(attr, Field)
    }

def __repr__(self: Any) -> str:
    kwargs = ', '.join(
        f'{key}={value!r}' for key, value in self._asdict().items()
    )
    return f'{self.__class__.__name__}({kwargs})'




# Recall that classes are instances of type. 
# These type hints strongly suggest this is a class decorator: 
# it takes a class and returns a class. 
def checked(cls: type) -> type:  
    for name, constructor in _fields(cls).items():    
        setattr(cls, name, Field(name, constructor)) 

    cls._fields = classmethod(_fields)  # type: ignore  

    # Module-level functions that will become instance methods of the decorated class. 
    instance_methods = (  
        __init__,
        __repr__,
        __setattr__,
        _asdict,
        __flag_unknown_attrs,
    )
    # Add each of the `instance_methods` to `cls`. 
    for method in instance_methods: 
        setattr(cls, method.__name__, method)

    # Return the decorated `cls`, 
    # fulfilling the essential contract of a class decorator. 
    return cls

In [9]:
@checked
class Movie:
    title: str
    year: int
    box_office: float

In [10]:
movie = Movie(title='The Godfather', year=1972, box_office=137)
movie.title

'The Godfather'

In [11]:
movie

Movie(title='The Godfather', year=1972, box_office=137.0)